In [1]:
import os

%pwd
os.chdir("../")
%pwd

'/home/arch_aditya/Desktop/text-summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path
    tokenizer_name : Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name,
        )

        return data_transformation_config


In [5]:
import os
from transformers import AutoTokenizer
from textSummarizer.logging import logger
from datasets import load_dataset , load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

/home/arch_aditya/Desktop/text-summarizer/TSZ/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-21 00:58:41,034 - INFO - config - 58 - PyTorch version 2.2.1 available.]


In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    logger.info(f"data transformation completed successfully")
except Exception as e:
    logger.error(e)

[2024-03-21 00:58:41,316 - INFO - common - 28 - yaml file : config/config.yaml loaded successfully]
[2024-03-21 00:58:41,318 - INFO - common - 28 - yaml file : params.yaml loaded successfully]
[2024-03-21 00:58:41,320 - INFO - common - 57 - Created directory artifacts]
[2024-03-21 00:58:41,321 - INFO - common - 57 - Created directory artifacts/data_transformation]
[2024-03-21 00:58:41,326 - DEBUG - connectionpool - 1055 - Starting new HTTPS connection (1): huggingface.co:443]
[2024-03-21 00:58:42,254 - DEBUG - connectionpool - 549 - https://huggingface.co:443 "HEAD /google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json HTTP/1.1" 200 0]
[2024-03-21 00:58:43,269 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_ingestion/samsum_dataset/dataset_dict.json]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

[2024-03-21 00:58:43,661 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_ingestion/samsum_dataset/train/tmplc2z85r3]


/home/arch_aditya/Desktop/text-summarizer/TSZ/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map:   0%|          | 0/819 [00:00<?, ? examples/s]

[2024-03-21 00:58:46,872 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_ingestion/samsum_dataset/test/tmpgzao9tdw]


...


Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]

[2024-03-21 00:58:47,354 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_transformation/samsum_dataset/validation/data-00000-of-00001.arrow]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 33469.33 examples/s]

[2024-03-21 00:58:47,378 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_transformation/samsum_dataset/validation/state.json]


[2024-03-21 00:58:47,385 - DEBUG - local - 298 - open file: /home/arch_aditya/Desktop/text-summarizer/artifacts/data_transformation/samsum_dataset/validation/dataset_info.json]
[2024-03-21 00:58:47,407 - INFO - 500070430 - 6 - data transformation completed successfully]
